In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from joblib import Parallel, delayed, parallel_backend
from tensorflow.keras.backend import clear_session

# Loading the Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Major Project/creditcard.csv")
df.head(5)

# Scale Amount and Time

In [ ]:
df[['Scaled_Time', 'Scaled_Amount']] = StandardScaler().fit_transform(df[['Time', 'Amount']])
df = df.drop(['Time', 'Amount'], axis=1)

# Handling Class Imbalance using SMOTE

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']
smote = SMOTE(sampling_strategy=0.2, random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
len(X_res)

# Split the Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=.2, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

# Training Isolation Forest Model

In [ ]:
iso_forest = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predicting the Values

In [ ]:
y_pred_train = iso_forest.predict(X_train)
y_pred_test = iso_forest.predict(X_test)

# Converting the Predictions

In [ ]:
y_pred_train = np.where(y_pred_train == 1, 0, 1)
y_pred_test = np.where(y_pred_test == 1, 0, 1)

# Evaluation of the Model using Metrics

In [ ]:
print('Classification report:\n', classification_report(y_test, y_pred_test))
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print('Confusion matrix:\n', cm)
print('Accuracy:', accuracy_score(y_test, y_pred_test))
precision = precision_score(y_test, y_pred_test, pos_label=1.0)
print('precision : ',precision)
recall = recall_score(y_test, y_pred_test, pos_label=1.0)
print('recall rate : ',recall)

# Saving the Model Metrics

In [ ]:
import os
from joblib import dump
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

# Define output directory
output_path = "/content/drive/MyDrive/Major Project/Model_Outputs"
os.makedirs(output_path, exist_ok=True)

# Saving the trained Isolation Forest model
model_save_path = os.path.join(output_path, "isolation_forest_model.joblib")
dump(iso_forest, model_save_path)
print("Model saved at:", model_save_path)

# Predicting the output
y_predicted = np.where(iso_forest.predict(X_test) == 1, 0, 1)  # 1 for normal, -1 for fraud mapped to binary classes

# Saving the classification report and confusion matrix
report = classification_report(y_test, y_predicted)
cm = confusion_matrix(y_true=y_test, y_pred=y_predicted)
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)

output_file = os.path.join(output_path, "iso_model_metrics.txt")
with open(output_file, "w") as f:
    f.write("Classification Report:\n")
    f.write(report)
    f.write("\nConfusion Matrix:\n")
    f.write(str(cm))
    f.write(f"\nAccuracy: {accuracy:.4f}")
    f.write(f"\nPrecision: {precision:.4f}")
    f.write(f"\nRecall: {recall:.4f}")
print(f"Metrics saved at: {output_file}")

# Saving confusion matrix as a plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
conf_matrix_path = os.path.join(output_path, 'confusion_matrix_iso.png')
plt.savefig(conf_matrix_path)
plt.close()
print(f"Confusion matrix plot saved at: {conf_matrix_path}")

# Loading the Saved Model

In [ ]:
import joblib
from joblib import load
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score, roc_auc_score,roc_curve,precision_recall_curve,auc,roc_curve,f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Path to load the saved model
model_load_path = "/content/drive/MyDrive/Major Project/Model_Outputs/isolation_forest_model.joblib"

# Load the Isolation Forest model
iso_forest_loaded = joblib.load(model_load_path)

# Predict on test data
y_pred_test = iso_forest_loaded.predict(X_test)
y_pred_test = np.where(y_pred_test == 1, 0, 1)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
accuracy=accuracy_score(y_test,y_pred_test)
f1=f1_score(y_test,y_pred_test)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# AUC-ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='red', label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.title("AUC-ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid()
plt.show()

# AUC-PRC Curve
precision_pr, recall_pr, _ = precision_recall_curve(y_test, y_pred_test)
pr_auc = auc(recall_pr, precision_pr)

plt.figure(figsize=(8, 6))
plt.plot(recall_pr, precision_pr, color='blue', lw=2, label=f'PR Curve (AUC = {pr_auc:.3f})')
plt.title("AUC-PRC Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower left")
plt.grid()
plt.show()